#Generate Data for GRU
This is the code used to generate data from the GRU generators 

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/My Drive/c147_project/')
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import time
from data import eegData
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
# get the device type of machine
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# These are the generator coes 

class GRUGenerator(nn.Module):
  """
  Generator that uses LSTM layer
  input is (batch, in_dim, seq)
  output is (batch, out_dim, seq)
  """
  def __init__(self, input_dim, hidden_dim, output_dim=22, n_layers=1, bias=False):
    super(GRUGenerator, self).__init__()
    self.input_dim = input_dim
    self.out_dim = output_dim
    self.hidden_dim = hidden_dim
    self.gru = nn.GRU(input_dim, hidden_dim, num_layers=n_layers, bias=bias, batch_first=True)

    self.linear = nn.Sequential(
        nn.Linear(hidden_dim, output_dim),
        nn.Tanh()
    )


  def forward(self, x):
    d1, d2, d3 = x.shape 
    x_reshaped = x.view(d1, d3, d2)
    recurrent, _ = self.gru(x_reshaped)
    out = self.linear(recurrent.contiguous().view(d1 * d3, self.hidden_dim))
    return out.view(d1, self.out_dim, d3)




In [ ]:
# generate the noise as well as fake labels
num_trials = 1000


In [ ]:
labels_store = torch.randint(0, 4, (num_trials, 1))
labels = labels_store.repeat(1, 100).reshape(num_trials, 1, -1)
labels.shape
sd = torch.load('Models/GRU-CNN/Generator_5000.pth')
GRU_CNN= GRUGenerator(4, 44)
GRU_CNN.load_state_dict(sd)
GRU_CNN.eval()
noise = torch.rand((num_trials, GRU_CNN.input_dim - 1, 100))
noise = torch.cat([noise, labels], dim=1)


fake_data = GRU_CNN(noise)

label_to_file = labels_store.numpy()
data_to_file = fake_data.detach().numpy()

np.save('Models/Fake_Data/GRU-CNN_data.npy', data_to_file)
np.save('Models/Fake_Data/GRU-CNN_labels.npy', label_to_file)

In [ ]:
labels_store = torch.randint(0, 4, (num_trials, 1))
labels =labels_store.repeat(1, 100).reshape(num_trials, 1, -1)
labels.shape

sd = torch.load('Models/GRU-LSTM/Generator_5000.pth')
GRU_LSTM= GRUGenerator(4, 44)
GRU_LSTM.load_state_dict(sd)
print(GRU_LSTM.eval())
noise = torch.rand((num_trials, GRU_LSTM.input_dim - 1, 100))
noise = torch.cat([noise, labels], dim=1)
noise.shape

fake_data = GRU_CNN(noise)

label_to_file = labels_store.numpy()
data_to_file = fake_data.detach().numpy()

np.save('Models/Fake_Data/GRU-LSTM_data.npy', data_to_file)
np.save('Models/Fake_Data/GRU-LSTM_labels.npy', label_to_file)

GRUGenerator(
  (gru): GRU(4, 44, bias=False, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=44, out_features=22, bias=True)
    (1): Tanh()
  )
)


In [ ]:
model_test = torch.load('Models_backup/GRU-LSTM/Generator_5000.pth', map_location=device)
sd = model_test.state_dict()

In [ ]:
model_real = GRUGenerator(4, 44)

In [ ]:
model_real.load_state_dict(sd)

<All keys matched successfully>